In [9]:
import graphviz
import pm4py
import os
log = pm4py.read_xes('example_log/example_log.xes')
from celonis_connect import Celonis_Connect
from pycelonis.celonis_api.pql.pql import PQL, PQLColumn, PQLFilter

parsing log, completed traces ::   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
cn = Celonis_Connect()
datamodel = cn.get_datamodels().find("rum_example")

/Users/baichaoye/opt/anaconda3/envs/lab/lib/python3.9/site-packages/pkg_resources/__init__.py:122: PkgResourcesDeprecationWarning: 1.7.1#sha256=1a33bea4a3fe5bf28b39faa561448594f768b243c3260800b7bddf5b80b46d2e is an invalid version and will not be supported in a future release
  warnings.warn(


[2022-07-03 21:48:38] INFO: Initial connect successful! Hello Baichao. PyCelonis Version: 1.7.1


In [13]:
table = cn.get_table().name
table

'example_log_xes'

In [6]:
from pm4py.algo.discovery.log_skeleton import algorithm as lsk_discovery
skeleton = lsk_discovery.apply(log, parameters={lsk_discovery.Variants.CLASSIC.value.Parameters.NOISE_THRESHOLD: 0.0})

In [23]:
activities = list(skeleton['activ_freq'].keys())
pql =PQL()

for act in activities:
    pql += PQLColumn(name = act,query = 'SUM(CALC_REWORK("'+table+'"."concept:name" IN (\''+act+'\')))')
df = datamodel.get_data_frame(pql)
df 

[2022-07-03 22:05:28] INFO: PQL Export started...
[2022-07-03 22:05:28] INFO: PQL Export status: DONE


,ER_Registration,IV_Antibiotics,ER_Triage,ER_Sepsis_Triage
0,3,3,3,17


In [24]:
freq_dic = {}
for key in df.columns:
    freq_dic[key] = df.loc[0,key]
freq_dic

{'ER_Registration': 3,
 'IV_Antibiotics': 3,
 'ER_Triage': 3,
 'ER_Sepsis_Triage': 17}

In [3]:
def visualize(skeleton,actv_log_freq):
    names=[]
    names.append(list(skeleton['equivalence']))
    names.append(list(skeleton['always_after']))
    names.append(list(skeleton['always_before']))
    names.append(list(skeleton['never_together']))
    names.append(list(skeleton['directly_follows']))
    freq_list = list(skeleton['activ_freq'])
    
    number_of_activities = len(names[0])+len(names[1])+len(names[2])+len(names[3])+len(names[4])
    
    f = graphviz.Digraph(filename = "output.gv")
    
    #initialize the list
    positions=[]    #node
    ns=[]      #name of node
    node_left=[]    #left node of the edge
    node_right=[]    #right node of the edge
    
    for i in range(len(names[0])): #equivalence
        node_left.append(names[0][i][0])
    for i in range(len(names[1])): #always_after
        node_left.append(names[1][i][0])
    for i in range(len(names[2])): #always_before
        node_left.append(names[2][i][0])
    for i in range(len(names[3])): #never_together
        node_left.append(names[3][i][0])
    for i in range(len(names[4])): #directly_follows
        node_left.append(names[4][i][0])
    
    
    #occurances of activities in any extended trace
    freq={}
    for k in range(number_of_activities):
        for j in range(len(freq_list)):
            if node_left[k] == freq_list[j]:
                freq[k] = skeleton['activ_freq'][list(skeleton['activ_freq'])[j]]
                #prepare for each node
                for i in range(len(names[0])): #equivalence
                    positions.append('{'+names[0][i][0]+'|{'+names[0][i][1]+'|'+str(actv_log_freq[i])+'|'+str(freq[k])+'}}')
                    ns.append(str(i))
                    node_right.append(names[0][i][1])
                for i in range(len(names[1])): #always_after
                    positions.append('{'+names[1][i][0]+'|{'+names[1][i][1]+'|'+str(actv_log_freq[i+len(names[0])])+'|'+str(freq[k])+'}}')
                    ns.append(str(len(names[0])+i))
                    node_right.append(names[1][i][1])
                for i in range(len(names[2])): #always_before
                    positions.append('{'+names[2][i][0]+'|{'+names[2][i][1]+'|'+str(actv_log_freq[i+len(names[0])+len(names[1])])+'|'+str(freq[k])+'}}')
                    ns.append(str(len(names[0])+len(names[1])+i))
                    node_right.append(names[2][i][1])
                for i in range(len(names[3])): #never_together
                    positions.append('{'+names[3][i][0]+'|{'+names[3][i][1]+'|'+str(actv_log_freq[i+len(names[0])+len(names[1])+len(names[2])])+'|'+str(freq[k])+'}}')
                    ns.append(str(len(names[0])+len(names[1])+len(names[2])+i))
                    node_right.append(names[3][i][1])
                for i in range(len(names[4])): #directly_follows
                    positions.append('{'+names[4][i][0]+'|{'+names[4][i][1]+'|'+str(actv_log_freq[i+len(names[0])+len(names[1])+len(names[2])+len(names[3])])+'|'+str(freq[k])+'}}')
                    ns.append(str(len(names[0])+len(names[1])+len(names[2])+len(names[3])+i))
                    node_right.append(names[4][i][1])

    #edges
    #equivalence, never_together: no edges
    for j in range(number_of_activities):
        for i in range(len(names[0]), len(names[0])+len(names[1])): #always_after
            # if the second activity is equal to the left activity of another relation
            #then there is a edge from the second relation to the first relation
            if (node_right[i] == node_left[j]):  
                f.edge(ns[j], ns[i],arrowtail='obox')
        for i in range(len(names[0])+len(names[1]), len(names[0])+len(names[1])+len(names[2])): #always_before
            if (node_right[i] == node_left[j]):  
                f.edge(ns[i], ns[j], arrowhead='obox',arrowtail='obox')
        for i in range(len(names[0])+len(names[1])+len(names[2])+len(names[3]), number_of_activities): #directly_follows
            if (node_right[i] == node_left[j]):  
                f.edge(ns[i], ns[j],arrowhead='odot')
    
    for name, position in zip(ns, positions):
        f.node(name, position, shape = "record",style = 'rounded')
    
    return(f)

In [4]:
actv_log_freq=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]

In [5]:
visualize(skeleton,actv_log_freq)

IndexError: list index out of range